In [1]:
# Install necessary libraries
!pip install pandas numpy scikit-learn sentence-transformers faiss-cpu joblib jupyterlab

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from scipy.sparse import hstack
import faiss
import joblib
import ast
import re
from math import sqrt

In [3]:
df_games = pd.read_csv('df_games.csv')

In [7]:
# Load the embeddings matrix from the .npy file
embeddings_matrix = np.load("embeddings_matrix.npy")
print("Embeddings matrix loaded successfully.")

Embeddings matrix loaded successfully.


In [4]:
#creating the search function
def search(dataframe, column_name, search_string):
    return dataframe[dataframe[column_name].str.contains(search_string, case=False, na=False)]

In [60]:
#keyword_soup cleaning
df_games['keyword_soup'] = df_games['keyword_soup'].fillna('').astype(str)

import re
# Replace multiple spaces with a single space and strip leading/trailing spaces
df_games['name'] = df_games['name'].str.replace(r'\s+', ' ', regex=True).str.strip()


In [61]:
df_games.to_csv("df_games.csv", index=False)

In [8]:
#vector creation (final_production_vectors.npy)
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import normalize
from scipy.sparse import hstack, csr_matrix


# --- 1. Define Your Final, Tuned Weights ---
# These weights prioritize content and publisher identity.
keyword_weight = 2.0
publisher_weight = 2.5
semantic_weight = 2.5
numerical_weight = 1.0

# --- 3. Build a Professional Preprocessing Pipeline ---

# Define the columns for each transformer
numerical_features = ['game_age', 'reviews_per_year', 'quality_score']
keyword_features = 'keyword_soup'
publisher_features = 'publisher_cleaned'

# Create the master preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('keywords', TfidfVectorizer(max_features=1000, stop_words='english', ngram_range=(1,2), min_df=5), keyword_features),
        ('publisher', TfidfVectorizer(max_features=500), publisher_features) # Limit publisher features
    ],
    transformer_weights={
        'num': numerical_weight,
        'keywords': keyword_weight,
        'publisher': publisher_weight
    })

# --- 4. Process Data and Combine with Semantics ---

# Apply the entire pipeline to get weighted numerical, keyword, and publisher features
processed_features = preprocessor.fit_transform(df_games)

# Apply weight to the semantic vectors
weighted_semantics = embeddings_matrix * semantic_weight

# Combine all features into one matrix
combined_weighted_vectors = hstack([
    processed_features,
    csr_matrix(weighted_semantics)
]).astype('float32')

# --- 5. Dimensionality Reduction for Accuracy ---

# Reduce the combined features to their most important 300 signals
svd = TruncatedSVD(n_components=300, random_state=42)
reduced_vectors = svd.fit_transform(combined_weighted_vectors)

# --- 6. Normalize and Finalize ---

# Normalize the final vectors for accurate similarity search
final_vectors_normalized = normalize(reduced_vectors, norm='l2', axis=1)

# Convert to the final format for Faiss
final_vectors = final_vectors_normalized.astype('float32')

print("Successfully created final, production-ready feature vectors.")
print("Final vector shape:", final_vectors.shape)

# Save the final vectors for your engine
np.save("final_production_vectors.npy", final_vectors)

Successfully created final, production-ready feature vectors.
Final vector shape: (88886, 300)


In [17]:
#engine 3 (the main engine file) (using IVFFlatL2)
import faiss
import numpy as np
import pandas as pd
#engine 3 (interactive recommendation engine)
import faiss
import numpy as np
import pandas as pd

# --- Load all necessary components ---
df_games = pd.read_csv("df_games.csv")  # Or your full cleaned dataset CSV
vectors = np.load("final_production_vectors.npy")  # Load the improved hybrid vectors

# Verify the dimensions of the vectors
print(f"Shape of vectors: {vectors.shape}")

# Rebuild the Faiss index with the correct dimensions
d = vectors.shape[1]  # Number of dimensions
index = faiss.IndexFlatL2(d)  # L2 distance metric
index.add(vectors)  # Add the vectors to the index

title_to_index = pd.Series(df_games.index, index=df_games['name'])

def get_profile_recommendations(game_titles, ratings, k=6):
    """
    Finds and prints recommendations based on a weighted average profile of input games.
    """
    try:
        # Check if the number of games and ratings match
        if len(game_titles) != len(ratings):
            print("Error: The number of games and ratings must be the same.")
            return []

        # 1. Check if all titles exist in the dataset
        missing_titles = [title for title in game_titles if title not in title_to_index.index]
        if missing_titles:
            print(f"Error: The following games were not found in the dataset: {', '.join(missing_titles)}")
            return []

        # 2. Get the vectors for all input games
        input_vectors = [vectors[title_to_index[title]] for title in game_titles]

        # 3. Calculate the WEIGHTED average vector to create the "taste profile"
        query_vector = np.average(input_vectors, axis=0, weights=ratings).reshape(1, -1).astype('float32')

        # Verify the dimensions of the query vector
        print(f"Shape of query vector: {query_vector.shape}")

        # Set nprobe for the loaded index
        index.nprobe = 10 

        # 4. Search the Faiss index
        distances, indices = index.search(query_vector, k)

        print(f"--- Recommendations for a fan of {', '.join(game_titles)} ---")
        
        # 5. Collect the results, filtering out the input games
        recs = []
        for i in range(k):
            rec_title = df_games.iloc[indices[0][i]]['name']
            if rec_title not in game_titles:
                recs.append(rec_title)

        return recs

    except KeyError as e:
        print(f"Error: Game {e} not found in the dataset.")
        return []

def interactive_recommendation_engine(initial_games, initial_ratings, threshold=8, max_recs=5):
    """
    Runs an interactive recommendation engine that refines recommendations based on user ratings.
    
    Args:
        initial_games (list): List of initial game titles.
        initial_ratings (list): List of ratings corresponding to the initial games.
        threshold (int): Minimum rating threshold for recommendations.
        max_recs (int): Maximum number of recommendations to finalize.
    """
    current_games = initial_games
    current_ratings = initial_ratings
    final_recommendations = []

    while len(final_recommendations) < max_recs:
        # Run the recommendation function
        recommendations = get_profile_recommendations(current_games, current_ratings, k=5)
        if not recommendations:
            print("No recommendations found. Exiting.")
            break

        # Ask the user to rate the recommendations
        print("\nRate the following recommendations (1-10):")
        new_games = []
        new_ratings = []

        for rec_game in recommendations:
            try:
                rating = int(input(f"Enter your rating for '{rec_game}' (1-10): "))
                if rating >= threshold:
                    new_games.append(rec_game)
                    new_ratings.append(rating)
            except ValueError:
                print("Invalid rating. Skipping this game.")

        # Update the list of games and ratings
        current_games = initial_games + new_games
        current_ratings = initial_ratings + new_ratings

        # Filter recommendations above the threshold
        final_recommendations = [game for game, rating in zip(current_games, current_ratings) if rating >= threshold]

        print(f"\nCurrent recommendations above threshold ({threshold}): {final_recommendations}")

        # Check if the user wants to stop
        stop = input("Press 'q' to stop or Enter to continue: ").strip().lower()
        if stop == 'q':
            break

    print("\nFinal recommendations:")
    for i, game in enumerate(final_recommendations[:max_recs], start=1):
        print(f"{i}. {game}")

# Example usage
test_games = ["Counter Strike 2", "Grand Theft Auto V", "Red Dead Redemption 2"]
test_ratings = [10, 10, 5]
interactive_recommendation_engine(test_games, test_ratings)
# --- Load all necessary components ---
df_games = pd.read_csv("df_games.csv")  # Or your full cleaned dataset CSV
vectors = np.load("final_production_vectors.npy")  # Load the improved hybrid vectors

# Verify the dimensions of the vectors
print(f"Shape of vectors: {vectors.shape}")

# Rebuild the Faiss index with the correct dimensions
d = vectors.shape[1]  # Number of dimensions
index = faiss.IndexFlatL2(d)  # L2 distance metric
index.add(vectors)  # Add the vectors to the index

title_to_index = pd.Series(df_games.index, index=df_games['name'])

def get_profile_recommendations(game_titles, ratings, k=6): # Added 'ratings' parameter
    """
    Finds and prints recommendations based on a weighted average profile of input games.
    """
    try:
        # Check if the number of games and ratings match
        if len(game_titles) != len(ratings):
            print("Error: The number of games and ratings must be the same.")
            return

        # 1. Check if all titles exist in the dataset
        missing_titles = [title for title in game_titles if title not in title_to_index.index]
        if missing_titles:
            print(f"Error: The following games were not found in the dataset: {', '.join(missing_titles)}")
            return
        
        # 2. Get the vectors for all input games
        input_vectors = [vectors[title_to_index[title]] for title in game_titles]
        
        # 3. Calculate the WEIGHTED average vector to create the "taste profile"
        query_vector = np.average(input_vectors, axis=0, weights=ratings).reshape(1, -1).astype('float32')
        
        # Verify the dimensions of the query vector
        print(f"Shape of query vector: {query_vector.shape}")

        # New line: Increase the search accuracy
        index.nprobe = 10 
        
        # 4. Search the Faiss index
        distances, indices = index.search(query_vector, k)
        
        print(f"--- Recommendations for a fan of {', '.join(game_titles)} ---")
        
        # 5. Print the results, filtering out the input games
        recs = []
        for i in range(k):
            rec_title = df_games.iloc[indices[0][i]]['name']
            if rec_title not in game_titles:
                recs.append(rec_title)
        
        for i, rec in enumerate(recs):
            print(f"{i+1}. {rec}")

    except KeyError as e:
        print(f"Error: Game {e} not found in the dataset.")

# --- Now, test with both games! ---
test_games = ["Counter Strike 2", "Grand Theft Auto V", "Red Dead Redemption 2"]
# Add a list of ratings (out of 10) to correspond to the games
test_ratings = [1, 10, 10]
# Call the function with the new ratings parameter
get_profile_recommendations(test_games, test_ratings)

Shape of vectors: (88886, 300)
Shape of query vector: (1, 300)
--- Recommendations for a fan of Counter Strike 2, Grand Theft Auto V, Red Dead Redemption 2 ---

Rate the following recommendations (1-10):

Current recommendations above threshold (8): ['Counter Strike 2', 'Grand Theft Auto V', 'Call of Duty', 'Marvel Rivals']
Shape of query vector: (1, 300)
--- Recommendations for a fan of Counter Strike 2, Grand Theft Auto V, Red Dead Redemption 2, Call of Duty, Marvel Rivals ---

Rate the following recommendations (1-10):

Current recommendations above threshold (8): ['Counter Strike 2', 'Grand Theft Auto V', 'Dying Light', 'HELLDIVERS 2']
Shape of query vector: (1, 300)
--- Recommendations for a fan of Counter Strike 2, Grand Theft Auto V, Red Dead Redemption 2, Dying Light, HELLDIVERS 2 ---

Rate the following recommendations (1-10):

Current recommendations above threshold (8): ['Counter Strike 2', 'Grand Theft Auto V', 'Marvel Rivals']
Shape of query vector: (1, 300)
--- Recommenda

In [ ]:
#create a recusive function to test the engine with different games and ratings

In [ ]:
#search(df_games, 'name', "assassin's creed")

In [6]:
#engine 3.1 (using IVFPQ)
import faiss
import numpy as np
import pandas as pd

# --- Load all necessary components ---
df_games = pd.read_csv("df_games.csv")  # Or your full cleaned dataset CSV
vectors = np.load("final_production_vectors.npy")  # Load the improved hybrid vectors

# Ensure vectors match the dimensionality of the Faiss index
if vectors.shape[1] < 304:
    padding_size = 304 - vectors.shape[1]
    print(f"Padding vectors: Adding {padding_size} dimensions to match Faiss index dimensionality.")
    vectors = np.hstack([vectors, np.zeros((vectors.shape[0], padding_size), dtype='float32')])
elif vectors.shape[1] > 304:
    print(f"Error: Vectors have more dimensions ({vectors.shape[1]}) than the Faiss index ({304}).")
    exit()

vectors = vectors.astype('float32')  # Convert to float32 if necessary
print("Adjusted shape of vectors:", vectors.shape)

# Load the pre-built Faiss index
try:
    index = faiss.read_index("production_games.index")  # Load the saved production index
    print("Successfully loaded production index.")
except FileNotFoundError:
    print("Error: production_games.index not found. Please ensure the index file exists.")
    exit()

# Verify the dimensions of the vectors and the Faiss index
print(f"Shape of vectors: {vectors.shape}")
print(f"Faiss index dimensionality: {index.d}")

# Ensure the Faiss index dimensionality matches the vectors
if vectors.shape[1] != index.d:
    print(f"Error: Dimensionality mismatch. Vectors have {vectors.shape[1]} dimensions, but the Faiss index expects {index.d}.")
    exit()

title_to_index = pd.Series(df_games.index, index=df_games['name'])

def get_profile_recommendations(game_titles, ratings, k=6): # Added 'ratings' parameter
    """
    Finds and prints recommendations based on a weighted average profile of input games.
    """
    try:
        # Check if the number of games and ratings match
        if len(game_titles) != len(ratings):
            print("Error: The number of games and ratings must be the same.")
            return

        # 1. Check if all titles exist in the dataset
        missing_titles = [title for title in game_titles if title not in title_to_index.index]
        if missing_titles:
            print(f"Error: The following games were not found in the dataset: {', '.join(missing_titles)}")
            return
        
        # 2. Get the vectors for all input games
        input_vectors = []
        for title in game_titles:
            if title in title_to_index:
                input_vectors.append(vectors[title_to_index[title]])
            else:
                print(f"Warning: '{title}' not found. Skipping.")

        if not input_vectors:
            print("Error: None of the input games were found.")
            return

        # 3. Calculate the WEIGHTED average vector to create the "taste profile"
        query_vector = np.average(input_vectors, axis=0, weights=ratings).reshape(1, -1).astype('float32')

        # Ensure the query vector is contiguous in memory
        query_vector = np.ascontiguousarray(query_vector, dtype='float32')

        # Verify the dimensions of the query vector
        print(f"Shape of query vector: {query_vector.shape}")

        # Set nprobe for the loaded index
        index.nprobe = 10 
        
        # 4. Search the Faiss index
        distances, indices = index.search(query_vector, k)
        
        print(f"--- Recommendations for a fan of {', '.join(game_titles)} ---")
        
        # 5. Print the results, filtering out the input games
        recs = []
        for i in range(k):
            rec_title = df_games.iloc[indices[0][i]]['name']
            if rec_title not in game_titles:
                recs.append(rec_title)
        
        for i, rec in enumerate(recs):
            print(f"{i+1}. {rec}")

    except KeyError as e:
        print(f"Error: Game {e} not found in the dataset.")

# --- Now, test with both games! ---
test_games = ["Counter Strike 2", "Grand Theft Auto V", "Red Dead Redemption 2"]
# Add a list of ratings (out of 10) to correspond to the games
test_ratings = [10, 10, 5]
# Call the function with the new ratings parameter
get_profile_recommendations(test_games, test_ratings)

Padding vectors: Adding 4 dimensions to match Faiss index dimensionality.
Adjusted shape of vectors: (88886, 304)
Successfully loaded production index.
Shape of vectors: (88886, 304)
Faiss index dimensionality: 304
Shape of query vector: (1, 304)
--- Recommendations for a fan of Counter Strike 2, Grand Theft Auto V, Red Dead Redemption 2 ---
1. Deep Rock Galactic
2. Halo Infinite
3. BattleBit Remastered
4. Dota 2
5. Tom Clancy's Rainbow Six Siege
6. Rust


In [4]:
import faiss
import numpy as np

# Load the final production vectors
vectors = np.load("final_production_vectors.npy")  # Ensure this file exists and contains the correct vectors
vectors = vectors.astype('float32')  # Convert to float32 if necessary
print("Original shape of vectors:", vectors.shape)

# Define parameters
d_original = vectors.shape[1]  # Current dimensionality
m = 8  # Number of sub-quantizers

# Check if the dimensionality is divisible by m
if d_original % m != 0:
    # Calculate the new dimensionality
    d_new = (d_original // m + 1) * m
    padding_size = d_new - d_original
    print(f"Padding vectors: Adding {padding_size} dimensions to make {d_new} divisible by {m}.")

    # Add padding
    padding = np.zeros((vectors.shape[0], padding_size), dtype='float32')
    vectors = np.hstack([vectors, padding])

print("Adjusted shape of vectors:", vectors.shape)

# Define dimensions and parameters
d = vectors.shape[1]  # Updated number of dimensions
nlist = 1024          # Number of clusters
nbits = 8             # Number of bits per sub-quantizer

# Build the index structure
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, nbits)

# Train the index
print("Training the production index...")
index.train(vectors)

# Add vectors to the index
print("Adding vectors to the index...")
index.add(vectors)

# Save the index
print("Saving index to 'production_games.index'...")
faiss.write_index(index, "production_games.index")

print("Production index created successfully.")

Original shape of vectors: (88886, 300)
Padding vectors: Adding 4 dimensions to make 304 divisible by 8.
Adjusted shape of vectors: (88886, 304)
Training the production index...
Adding vectors to the index...
Saving index to 'production_games.index'...
Production index created successfully.


In [ ]:
#engine 3 (not to be touched)

import faiss
import numpy as np
import pandas as pd

# --- Load all necessary components ---
df_games = pd.read_csv("df_games.csv")  # Or your full cleaned dataset CSV
vectors = np.load("final_production_vectors.npy")  # Load the improved hybrid vectors

# Verify the dimensions of the vectors
print(f"Shape of vectors: {vectors.shape}")

# Rebuild the Faiss index with the correct dimensions
d = vectors.shape[1]  # Number of dimensions
index = faiss.IndexFlatL2(d)  # L2 distance metric
index.add(vectors)  # Add the vectors to the index

title_to_index = pd.Series(df_games.index, index=df_games['name'])

def get_profile_recommendations(game_titles, ratings, k=6): # Added 'ratings' parameter
    """
    Finds and prints recommendations based on a weighted average profile of input games.
    """
    try:
        # Check if the number of games and ratings match
        if len(game_titles) != len(ratings):
            print("Error: The number of games and ratings must be the same.")
            return

        # 1. Check if all titles exist in the dataset
        missing_titles = [title for title in game_titles if title not in title_to_index.index]
        if missing_titles:
            print(f"Error: The following games were not found in the dataset: {', '.join(missing_titles)}")
            return
        
        # 2. Get the vectors for all input games
        input_vectors = [vectors[title_to_index[title]] for title in game_titles]
        
        # 3. Calculate the WEIGHTED average vector to create the "taste profile"
        query_vector = np.average(input_vectors, axis=0, weights=ratings).reshape(1, -1).astype('float32')
        
        # Verify the dimensions of the query vector
        print(f"Shape of query vector: {query_vector.shape}")

        # New line: Increase the search accuracy
        index.nprobe = 10 
        
        # 4. Search the Faiss index
        distances, indices = index.search(query_vector, k)
        
        print(f"--- Recommendations for a fan of {', '.join(game_titles)} ---")
        
        # 5. Print the results, filtering out the input games
        recs = []
        for i in range(k):
            rec_title = df_games.iloc[indices[0][i]]['name']
            if rec_title not in game_titles:
                recs.append(rec_title)
        
        for i, rec in enumerate(recs):
            print(f"{i+1}. {rec}")

    except KeyError as e:
        print(f"Error: Game {e} not found in the dataset.")

    

# --- Now, test with both games! ---
test_games = ["EA SPORTS FC 25", "Grand Theft Auto V", "Red Dead Redemption 2"]
# Add a list of ratings (out of 10) to correspond to the games
test_ratings = [1, 2, 10]
# Call the function with the new ratings parameter
get_profile_recommendations(test_games, test_ratings)

Shape of vectors: (88886, 300)
Shape of query vector: (1, 300)
--- Recommendations for a fan of EA SPORTS FC 25, Grand Theft Auto V, Red Dead Redemption 2 ---
1. Cyberpunk 2077
2. Apex Legends
3. Baldur's Gate 3
4. 7 Days to Die
5. Sons Of The Forest


In [2]:
# --- Now, test with both games! ---
test_games = ["Marvel's Spider Man 2", "Grand Theft Auto V", "Red Dead Redemption 2"]
# Add a list of ratings (out of 10) to correspond to the games
test_ratings = [10, 10, 10]
# Call the function with the new ratings parameter
get_profile_recommendations(test_games, test_ratings)

Shape of query vector: (1, 300)
--- Recommendations for a fan of Marvel's Spider Man 2, Grand Theft Auto V, Red Dead Redemption 2 ---
1. Mafia II Classic
2. V Rising
3. Ready or Not
4. Path of Exile 2
5. Kingdom Come Deliverance II


In [11]:
# --- Now, test with both games! ---
test_games = ["Cyberpunk 2077", "Apex Legends", "Sons Of The Forest", "EA SPORTS FC 25", "Grand Theft Auto V", "Red Dead Redemption 2"]
# Add a list of ratings (out of 10) to correspond to the games
test_ratings = [1, 1, 2, 2, 8, 8]
# Call the function with the new ratings parameter
get_profile_recommendations(test_games, test_ratings)

Shape of query vector: (1, 300)
--- Recommendations for a fan of Cyberpunk 2077, Apex Legends, Sons Of The Forest, EA SPORTS FC 25, Grand Theft Auto V, Red Dead Redemption 2 ---
1. My Summer Car
2. Call of Duty
3. 7 Days to Die
4. Marvel Rivals


In [13]:
import faiss
import numpy as np
import pandas as pd

# --- Load all necessary components ---
df_games = pd.read_csv("df_games.csv")  # Or your full cleaned dataset CSV
vectors = np.load("final_production_vectors.npy")  # Load the improved hybrid vectors

# Verify the dimensions of the vectors
print(f"Shape of vectors: {vectors.shape}")

# Rebuild the Faiss index with the correct dimensions
d = vectors.shape[1]  # Number of dimensions
index = faiss.IndexFlatL2(d)  # L2 distance metric
index.add(vectors)  # Add the vectors to the index

title_to_index = pd.Series(df_games.index, index=df_games['name'])

def get_profile_recommendations(game_titles, ratings, k=6):
    """
    Finds and prints recommendations based on a weighted average profile of input games.
    """
    try:
        # Check if the number of games and ratings match
        if len(game_titles) != len(ratings):
            print("Error: The number of games and ratings must be the same.")
            return []

        # 1. Check if all titles exist in the dataset
        missing_titles = [title for title in game_titles if title not in title_to_index.index]
        if missing_titles:
            print(f"Error: The following games were not found in the dataset: {', '.join(missing_titles)}")
            return []

        # 2. Get the vectors for all input games
        input_vectors = [vectors[title_to_index[title]] for title in game_titles]

        # 3. Calculate the WEIGHTED average vector to create the "taste profile"
        query_vector = np.average(input_vectors, axis=0, weights=ratings).reshape(1, -1).astype('float32')

        # Verify the dimensions of the query vector
        print(f"Shape of query vector: {query_vector.shape}")

        # Set nprobe for the loaded index
        index.nprobe = 10 
        
        # 4. Search the Faiss index
        distances, indices = index.search(query_vector, k)

        print(f"--- Recommendations for a fan of {', '.join(game_titles)} ---")
        
        # 5. Collect the results, filtering out the input games
        recs = []
        for i in range(k):
            rec_title = df_games.iloc[indices[0][i]]['name']
            if rec_title not in game_titles:
                recs.append(rec_title)

        for i, rec in enumerate(recs):
            print(f"{i+1}. {rec}")

        return recs

    except KeyError as e:
        print(f"Error: Game {e} not found in the dataset.")
        return []

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

# --- Save the index to a file ---
print("\nSaving index to 'games_index.index'...")
faiss.write_index(index, "games_index.index")
print("Index saved successfully.")

Shape of vectors: (88886, 300)

Saving index to 'games_index.index'...
Index saved successfully.


In [7]:
search(df_games, 'name', "Ready or not")

,appid,name,release_date,about_the_game,short_description,windows,mac,linux,recommendations,publishers,...,ps5,xbox,tags_cleaned,combined,game_age,reviews_per_year,quality_score,keyword_soup,publisher_cleaned,combined_descriptions
89,1144200,Ready or Not,2023-12-13,Be sure to join the Ready or Not Discord serve...,"Ready or Not is an intense, tactical, first-pe...",True,False,False,176271,['VOID Interactive'],...,False,False,Tactical Tactical Tactical Tactical Tactical T...,"['Single-player', 'Multi-player', 'Co-op', 'On...",2,58920,0.888533,Tactical Tactical Tactical Tactical Tactical T...,['voidinteractive'],"Ready or Not is an intense, tactical, first-pe..."
